In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from eugenenet.vgg11 import VGG11
from eugenenet.smallvggnet import SmallVGGNet
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
#from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os



Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/font_manager.py:279: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [9]:
import pandas as pd

# initialize the data and labels
print("[INFO] loading images...")
#data = []
#labels = []
NUMBER_OF_SAMPLES = 100000

TRAIN_DATA_FILE = 'synimg/train/data.csv'
# grab the image paths and randomly shuffle them
df_imagePaths = pd.DataFrame.from_csv(TRAIN_DATA_FILE)
df_imagePaths_sample = df_imagePaths.sample(n=NUMBER_OF_SAMPLES)


[INFO] loading images...


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:11: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [10]:
# construct the image generator for data augmentation
#aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
#	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,#
#	horizontal_flip=True, fill_mode="nearest")
 
datagen=ImageDataGenerator(rescale=1./255., validation_split=0.15)
    


In [11]:
train_generator=datagen.flow_from_dataframe(
dataframe=df_imagePaths_sample,
directory="./",
x_col="filepath",
y_col="style_name",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,64))

valid_generator=datagen.flow_from_dataframe(
dataframe=df_imagePaths_sample,
directory=".",
x_col="filepath",
y_col="style_name",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,64))

Found 85000 validated image filenames belonging to 10 classes.
Found 15000 validated image filenames belonging to 10 classes.


In [12]:
# initialize our VGG-like Convolutional Neural Network
model = VGG11.build(width=64, height=32, depth=3, classes=len(train_generator.class_indices))

# initialize our initial learning rate, # of epochs to train for,
# and batch size
INIT_LR = 0.01
EPOCHS = 75
BS = 128
 
# initialize the model and optimizer (you'll want to use
# binary_crossentropy for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
 
# train the network
#H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS, epochs=EPOCHS)

#H = model.fit(trainX, trainY, steps_per_epoch=len(trainX) // BS, epochs=EPOCHS)

[INFO] training network...


In [13]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

H = model.fit_generator(generator = train_generator, 
                        steps_per_epoch=STEP_SIZE_TRAIN, 
                        validation_data = valid_generator,  
                        validation_steps=STEP_SIZE_VALID, 
                        epochs=EPOCHS, 
                        callbacks=[
                            EarlyStopping(patience=5),
                            ReduceLROnPlateau(patience=3)
                        ]
                       )

Epoch 1/75
2656/2656 [==============================] - 55s 21ms/step - loss: 0.9619 - acc: 0.6263 - val_loss: 0.8035 - val_acc: 0.6556
Epoch 2/75
2656/2656 [==============================] - 54s 20ms/step - loss: 0.7467 - acc: 0.6824 - val_loss: 0.7553 - val_acc: 0.6728
Epoch 3/75
2656/2656 [==============================] - 54s 20ms/step - loss: 0.6796 - acc: 0.7108 - val_loss: 0.6543 - val_acc: 0.7181
Epoch 4/75
2656/2656 [==============================] - 54s 20ms/step - loss: 0.6026 - acc: 0.7524 - val_loss: 0.4946 - val_acc: 0.7985
Epoch 5/75
2656/2656 [==============================] - 54s 20ms/step - loss: 0.4996 - acc: 0.8024 - val_loss: 0.4124 - val_acc: 0.8311
Epoch 6/75
2656/2656 [==============================] - 54s 20ms/step - loss: 0.4149 - acc: 0.8353 - val_loss: 0.3635 - val_acc: 0.8497
Epoch 7/75
2656/2656 [==============================] - 54s 20ms/step - loss: 0.3660 - acc: 0.8544 - val_loss: 0.2992 - val_acc: 0.8769
Epoch 8/75
2656/2656 [==========================

With batch normalisation: 
* 40 epochs: acc: 0.6613, loss: 0.7873
* 65 epochs: acc: ~0.70
* 75 epochs: acc: 0.7248


In [14]:
len(train_generator.class_indices)

10

In [15]:
model.layers

In [16]:
model.evaluate_generator(generator= valid_generator,steps=STEP_SIZE_VALID)

[0.1838087858731548, 0.92083333333333328]

In [17]:
valid_generator.class_indices

{'Beijing': 0,
 'Brisbane': 1,
 'Geneva': 2,
 'HongKong': 3,
 'Luanda': 4,
 'Melbourne': 5,
 'Seoul': 6,
 'Singapore': 7,
 'Sydney': 8,
 'Zurich': 9}

In [18]:
train_generator.class_indices

{'Beijing': 0,
 'Brisbane': 1,
 'Geneva': 2,
 'HongKong': 3,
 'Luanda': 4,
 'Melbourne': 5,
 'Seoul': 6,
 'Singapore': 7,
 'Sydney': 8,
 'Zurich': 9}

In [14]:
model_save_name = "./trained_models/vgg11_v2"
model.save_weights('{0}.h5'.format(model_save_name))

In [20]:
model_save_name = "./trained_models/"
print('{0}.h5'.format(model_save_name))

./trained_models/.h5


In [15]:
TEST_DATA_FILE = 'synimg/test/data_nostyle.csv'
df_test = pd.read_csv(TEST_DATA_FILE)

In [22]:
df_test

,id,file,filepath
0,9000000,test-A-9000000.jpg,synimg/test/A/test-A-9000000.jpg
1,9000001,test-B-9000001.jpg,synimg/test/B/test-B-9000001.jpg
2,9000002,test-C-9000002.jpg,synimg/test/C/test-C-9000002.jpg
3,9000003,test-D-9000003.jpg,synimg/test/D/test-D-9000003.jpg
4,9000004,test-E-9000004.jpg,synimg/test/E/test-E-9000004.jpg
5,9000005,test-F-9000005.jpg,synimg/test/F/test-F-9000005.jpg
6,9000006,test-G-9000006.jpg,synimg/test/G/test-G-9000006.jpg
7,9000007,test-H-9000007.jpg,synimg/test/H/test-H-9000007.jpg
8,9000008,test-I-9000008.jpg,synimg/test/I/test-I-9000008.jpg
9,9000009,test-J-9000009.jpg,synimg/test/J/test-J-9000009.jpg


In [16]:
testgen=ImageDataGenerator(rescale=1./255.)

test_generator=testgen.flow_from_dataframe(
dataframe=df_test,
directory="./",
x_col="filepath",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(32,64))


STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()

pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

Found 20000 validated image filenames.
625/625 [==============================] - 6s 9ms/step


In [17]:
predicted_class_indices=np.argmax(pred,axis=1)

In [18]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [26]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)

In [19]:
df_test['style_name'] = predictions

In [20]:
df_test.to_csv('submission_vgg11_v2.csv', columns=['id','style_name'], index = False)

In [50]:
df_test

,id,file,filepath,style_name
0,9000000,test-A-9000000.jpg,synimg/test/A/test-A-9000000.jpg,Geneva
1,9000001,test-B-9000001.jpg,synimg/test/B/test-B-9000001.jpg,Melbourne
2,9000002,test-C-9000002.jpg,synimg/test/C/test-C-9000002.jpg,Melbourne
3,9000003,test-D-9000003.jpg,synimg/test/D/test-D-9000003.jpg,Singapore
4,9000004,test-E-9000004.jpg,synimg/test/E/test-E-9000004.jpg,Brisbane
5,9000005,test-F-9000005.jpg,synimg/test/F/test-F-9000005.jpg,Singapore
6,9000006,test-G-9000006.jpg,synimg/test/G/test-G-9000006.jpg,HongKong
7,9000007,test-H-9000007.jpg,synimg/test/H/test-H-9000007.jpg,Sydney
8,9000008,test-I-9000008.jpg,synimg/test/I/test-I-9000008.jpg,Sydney
9,9000009,test-J-9000009.jpg,synimg/test/J/test-J-9000009.jpg,Singapore


In [13]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1',
 '/job:localhost/replica:0/task:0/device:GPU:2',
 '/job:localhost/replica:0/task:0/device:GPU:3']